In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.api as sm
from sklearn import linear_model
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.preprocessing import StandardScaler

import os

from sklearn.model_selection import train_test_split

In [4]:
import openpyxl

In [11]:
df = pd.read_excel('ygl_plan.xlsx',header=1,engine='openpyxl')

In [20]:
pd.set_option('display.max_rows',400)

In [23]:
df.drop_duplicates(subset='훈련일자',inplace=True)

In [26]:
df.rename(columns= {"훈련일자":"date","교과목(및 능력단위)코드":"content"},inplace=True)

In [28]:
df = df[['date','content']]

In [34]:
df = df[df.date>20210113]

In [38]:
df['content'] = df.content.str.replace('_[0-9]+','')

C:\Users\epsilon\dev\Anaconda\envs\fortf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [59]:
df['content'] = df.content.str.replace('_',' ')

C:\Users\epsilon\dev\Anaconda\envs\fortf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
df.to_csv('clean_plan.csv')

In [62]:
df

,date,content
57,20210114,기계학습 Ensemble기법
60,20210115,기계학습 강화학습
63,20210118,기계학습 강화학습2
66,20210119,딥러닝 기본 딥러닝 이해
69,20210120,딥러닝 기본 DNN
72,20210121,딥러닝 기본 DNN
75,20210122,딥러닝 기본 CNN
78,20210125,딥러닝 기본 CNN
81,20210126,딥러닝 기본 RNN
84,20210127,딥러닝 기본 RNN
